In [156]:
import os
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup

In [157]:
SCORE_DIR = "data/scores"

In [158]:
box_scores = os.listdir(SCORE_DIR)

In [159]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [160]:
box_scores

['data/scores/202211070MIA.html',
 'data/scores/202212140DAL.html',
 'data/scores/202212070TOR.html',
 'data/scores/202402130BRK.html',
 'data/scores/202311110ATL.html',
 'data/scores/202212090UTA.html',
 'data/scores/202404110POR.html',
 'data/scores/202210290BRK.html',
 'data/scores/202310280CLE.html',
 'data/scores/202211070DET.html',
 'data/scores/202401050PHI.html',
 'data/scores/202305060MIA.html',
 'data/scores/202212060DEN.html',
 'data/scores/202211190POR.html',
 'data/scores/202212070MEM.html',
 'data/scores/202211140GSW.html',
 'data/scores/202403250HOU.html',
 'data/scores/202403140MIL.html',
 'data/scores/202403220POR.html',
 'data/scores/202302260GSW.html',
 'data/scores/202406060BOS.html',
 'data/scores/202210290CHI.html',
 'data/scores/202403170WAS.html',
 'data/scores/202403230POR.html',
 'data/scores/202304260MIL.html',
 'data/scores/202212140ORL.html',
 'data/scores/202311220PHO.html',
 'data/scores/202312110HOU.html',
 'data/scores/202304090NYK.html',
 'data/scores/

In [161]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return str(soup)

In [162]:
def parse_html_as_soup(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [163]:
def read_line_score(soup):
    line_score_io = StringIO(soup)
    line_score = pd.read_html(line_score_io, attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team", "total"]]

    return line_score

In [164]:
def read_stats(soup, team, stat):
    df = pd.read_html(StringIO(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [165]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [166]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    soup_as_soup = parse_html_as_soup(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])
    
    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
    
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
    
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
    
        summary = pd.concat([totals, maxes])
    
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
    
        summary = summary[base_cols]
    
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T
    
    game = pd.concat([summary, line_score], axis=1)
    
    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"
    
    full_game = pd.concat([game, game_opp], axis=1)
    
    full_game["season"] = read_season_info(soup_as_soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 2639
200 / 2639
300 / 2639
400 / 2639
500 / 2639
600 / 2639
700 / 2639
800 / 2639
900 / 2639
1000 / 2639
1100 / 2639
1200 / 2639
1300 / 2639
1400 / 2639
1500 / 2639
1600 / 2639
1700 / 2639
1800 / 2639
1900 / 2639
2000 / 2639
2100 / 2639
2200 / 2639
2300 / 2639
2400 / 2639
2500 / 2639
2600 / 2639


In [167]:
games_df = pd.concat(games, ignore_index=True)

In [168]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,37.0,87.0,0.425,7.0,33.0,0.212,32.0,35.0,...,20.0,32.3,226.0,117.0,TOR,126,1,2023,2022-12-07,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5273,240.0,240.0,47.0,89.0,0.528,14.0,31.0,0.452,19.0,25.0,...,41.0,45.3,162.0,135.0,DEN,126,0,2023,2022-12-28,True
5274,240.0,240.0,43.0,87.0,0.494,10.0,27.0,0.370,21.0,27.0,...,33.6,31.0,163.0,123.0,SAS,110,1,2024,2023-11-10,True
5275,240.0,240.0,43.0,92.0,0.467,12.0,34.0,0.353,12.0,19.0,...,51.5,33.0,200.0,112.0,MIN,117,0,2024,2023-11-10,False
5276,240.0,240.0,42.0,82.0,0.512,12.0,27.0,0.444,12.0,19.0,...,53.2,28.0,146.0,126.0,TOR,121,1,2024,2023-11-24,False


In [170]:
games_df.to_csv("nba_games.csv")